In [51]:
# Create kaldi features

In [52]:
from bob import kaldi
import scipy.io.wavfile
import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm_notebook as tqdm

In [53]:
def load_wav(audio_path):
    rate, audio_sample = scipy.io.wavfile.read(audio_path)
    audio_sample = np.cast['float64'](audio_sample)
    return audio_sample, rate

def compute_mfcc(audio_sample, rate):
    """ (? actually mfcc(audio_sample)=mfcc(K*audio_sample)) Note: the input needs to be normalized between [-1, 1] """
    mfcc = kaldi.cepstral(
       audio_sample, cepstral_type='mfcc', rate=rate, 
       raw_energy=False, 
       num_mel_bins=40,
       num_ceps=40,
       low_freq=20, high_freq=-400,
       delta_order=0
    )
    return mfcc


def get_mfcc(audio):
    return compute_mfcc(*load_wav(audio))

def save_to_text(file_name, mfcc):
    with open(file_name, 'w') as f:
        np.savetxt(f, mfcc, delimiter=', ', newline='], [', fmt='%f')
    a = ''
    with open(file_name, 'r') as f:
        for l in f:
            a = '[['+l[:-4]+']]'
    with open(file_name, 'w') as f:
        f.write(a)

In [54]:
source_path = Path('wav/')
target_path = Path('kaldi/norm/')

In [55]:
for wav in tqdm(source_path.iterdir(), total=1681):
    mfcc = get_mfcc(wav)
    save_to_text(target_path / wav.stem, mfcc)